In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save
import datetime
import dateutil.parser
from os.path import join

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    get_visualization_subtitle,
    get_country_color_map,
)
from theme import apply_theme
from web import for_website

alt.data_transformers.disable_max_rows(); # Allow using rows more than 5000

In [ ]:
data_release='2021-04-29'

consistent_date = {
    '2020-Mar-Apr': "'20 Mar - '20 Apr",
    '2020-May-Jun': "'20 May - '20 Jun",
    '2020-Jul-Aug': "'20 Jul - '20 Aug",
    '2020-Sep-Oct': "'20 Sep - '20 Oct",
    '2020-Nov-2021-Jan': "'20 Nov - '21 Jan"
}

date = ['Mar - Apr', 'May - Jun', 'Jul - Aug', 'Sep - Oct', 'Since Nov']
date = ["'20 Mar - '20 Apr", "'20 May - '20 Jun", "'20 Jul - '20 Aug", "'20 Sep - '20 Oct", "'20 Nov - '21 Jan"]

sites = ['META', 'APHP', 'FRBDX', 'ICSM', 'NWU', 'BIDMC', 'MGB', 'UCLA', 'UMICH', 'UPENN', 'UPITT', 'VA1', 'VA2', 'VA3', 'VA4', 'VA5']
site_colors = ['black', '#D45E00', '#0072B2', '#CB7AA7', '#E79F00', '#029F73', '#DBD03C', '#57B4E9', '#57B4E9', '#57B4E9', '#57B4E9', '#57B4E9']

df = pd.read_csv(join("..", "data", "Phase2.1SurvivalRSummariesPublic", "ToShare", "table.score.toShare.csv"))
print(df.head())

# Rename columns
df = df.drop(columns=["Unnamed: 0"])
df = df.rename(columns={
    'siteid': 'site',
    'calendar_month': 'month'
})

# More readable values
df.site = df.site.apply(lambda x: x.upper())

print(df.site.unique().tolist())
print(df.month.unique().tolist())

# Drop "combine" sites
df = df[df.site != "COMBINE"]

# df = pd.melt(df, id_vars=['siteid'], value_vars=date, var_name='date', value_name='value')

df.month = df.month.apply(lambda x: consistent_date[x])

# Add a reference (META)
# df['reference'] = df.date.apply(lambda x: df[(df.date == x) & (df.siteid == 'META')].value.sum())

df.head()

In [ ]:
def risk(_d, metric='pos'):
    d = _d.copy()
    
    """
    DATA PREPROCESSING...
    """
    d.loc[d.site == 'combine', 'site'] = 'All Sites'
    d.cat = d.cat.apply(lambda x: {
        'L':'Low Risk', 
        'M': 'Medium Risk', 
        'H': 'High Risk',
        'H/M': 'High/Medium',
        'L/M': 'Low/Medium'
    }[x])
    
    """
    PLOT!
    """
    y_title = '% of Patients in Each Category' if metric == 'pos' else '% of Event in Each Category'
    colors = ['#7BADD1', '#427BB5', '#14366E'] if metric == 'pos' else ['#A8DED1', '#3FA86F', '#005A24'] if metric == 'ppv' else ['red', 'salmon']
    colorDomain = ['High/Medium', 'Low/Medium'] if metric == 'rr' else ['Low Risk', 'Medium Risk', 'High Risk']
    width = 300
    size = 50
    y_scale = alt.Scale(domain=[0, 1]) if metric == 'pos' or metric=='ppv' else alt.Scale()
    
    bar = alt.Chart(
        d
    ).transform_calculate(
        order="{'Low Risk':0, 'Medium Risk': 1, 'High Risk': 2}[datum.variable]"  
    ).transform_filter(
        {'field': 'metric', 'oneOf': [metric]}
    ).encode(
        x=alt.X("month:N", title='Month', scale=alt.Scale(domain=['Mar-Apr', 'May-Jun', 'Jul-Aug', 'Sep-Oct', 'Since Nov'])),
        y=alt.Y("value:Q", title=y_title, axis=alt.Axis(format='.0%'), scale=y_scale),
        color=alt.Color("cat:N", title='Category', scale=alt.Scale(domain=colorDomain, range=colors)),
        order="order:O"
    ).properties(
        width=width
    )
    
    if metric == 'pos':
        bar = bar.mark_bar(
            size=size, stroke='black'
        )
    else:
        bar = bar.mark_line(
            size=3, point=True, opacity=0.8
        )
    
    d['visibility'] = d['value'] > 0.08
    text = alt.Chart(
        d
    ).transform_filter(
        {'field': 'metric', 'oneOf': [metric]}
    ).mark_text(size=16, dx=0, dy=5, color='white', baseline='top', fontWeight=500).encode(
        x=alt.X('month:N'),
        y=alt.Y('value:Q', stack='zero'),
        detail='cat:N',
        text=alt.Text('value:Q', format='.0%'),
        order="order:O",
        opacity=alt.Opacity('visibility:N', scale=alt.Scale(domain=[True, False], range=[1, 0]))
    )
#     .transform_filter(
#         (f'datum.value > 0.10')
#     )
    
    if metric == 'pos':
        bar = (bar + text)
    
    bar = bar.facet(
        column=alt.Column('site:N', header=alt.Header(title=None)),
    )
    
    """
    COMBINE
    """
    res = bar.properties(
        title={
            "text": [
                f"Distribution of Risk Scores" if metric == 'pos' else f"Event Rate of Risk Scores"
            ],
            "dx": 80,
            "subtitle": [
                # lab, #.title(),
                get_visualization_subtitle(data_release=data_release, with_num_sites=False)
            ], 
            "subtitleColor": "gray",
        }
    )

    
    return res

In [ ]:
d = df.copy()

width = 160
height = 180
height2 = 140
size = 28
point=alt.OverlayMarkDef(filled=False, fill='white', strokeWidth=2)


"""
DATA PREPROCESSING...
"""
d.loc[d.site == 'combine', 'site'] = 'All Sites'
d.cat = d.cat.apply(lambda x: {
    'L':'Low', 
    'M': 'Medium', 
    'H': 'High',
    'H/M': 'H/M',
    'L/M': 'L/M'
}[x])



"""
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%% TOP %%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
"""
metric='pos'

y_title = '% of Patients in Each Category'
y_scale = alt.Scale(domain=[0, 1])
colors = ['#7BADD1', '#427BB5', '#14366E']
# colorDomain = ['Low Risk', 'Medium Risk', 'High Risk']
colorDomain = ['Low', 'Medium', 'High']
# colorDomain = ['L', 'M', 'H']

bar = alt.Chart(
    d
).transform_calculate(
    order="{'L':0, 'M': 1, 'H': 2}[datum.variable]"  
).transform_filter(
    {'field': 'metric', 'oneOf': [metric]}
).encode(
    x=alt.X("month:N", title=None, axis=alt.Axis(labelAngle=-55), sort=date),
    y=alt.Y("value:Q", title=y_title, axis=alt.Axis(format='.0%'), scale=y_scale),
    color=alt.Color("cat:N", title='Risk Level', scale=alt.Scale(domain=colorDomain, range=colors)),
    order="order:O"
).properties(
    width=width,
    height=height
)

bar = bar.mark_bar(
    size=size, stroke='black'
)


d['visibility'] = d['value'] > 0.08

text = alt.Chart(
    d
).transform_filter(
    {'field': 'metric', 'oneOf': [metric]}
).mark_text(size=14, dx=0, dy=5, color='white', baseline='top', fontWeight=500).encode(
    x=alt.X("month:N", title=None, axis=alt.Axis(labelAngle=-55), sort=date),
    y=alt.Y('value:Q', stack='zero'),
    detail='cat:N',
    text=alt.Text('value:Q', format='.0%'),
    order="order:O",
    opacity=alt.Opacity('visibility:N', scale=alt.Scale(domain=[True, False], range=[1, 0]), legend=None)
)

if metric == 'pos':
    bar = (bar + text)

bar = bar.facet(
    column=alt.Column('site:N', header=alt.Header(title=None), sort=sites),
    spacing=20
)

"""
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%% Bottom %%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
"""
metric='ppv'

y_title = '% of Event'
colors = ['#A8DED1', '#3FA86F', '#005A24']
colors = ['#00A87E', '#00634B', 'black']
y_scale = alt.Scale(domain=[0, 1])

line = alt.Chart(
    d
).transform_calculate(
#     order="{'Low Risk':0, 'Medium Risk': 1, 'High Risk': 2}[datum.variable]"  
    order="{'L':0, 'M': 1, 'H': 2}[datum.variable]"  
).transform_filter(
    {'field': 'metric', 'oneOf': [metric]}
).encode(
    # x=alt.X("month:N", title=None, axis=alt.Axis(labelAngle=-55), sort=date),
    x=alt.X("month:N", title='Month', scale=alt.Scale(domain=date), axis=alt.Axis(grid=False, ticks=False, labels=False, domain=False, title=None, labelAngle=-55)),
    y=alt.Y("value:Q", title=y_title, scale=y_scale),
    color=alt.Color("cat:N", title='Risk Level', scale=alt.Scale(domain=colorDomain, range=colors)),
    order="order:O"
).properties(
    width=width,
    height=height2
)


line = line.mark_line(
    size=3, point=point, opacity=0.8
).facet(
    column=alt.Column('site:N', header=alt.Header(title=None), sort=sites),
)



"""
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%% Bottom 2 %%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
"""
metric='rr'
y_title = 'Ratio Between Risk Levels'
# colors = ['red', 'salmon']
colors = ['#D45E00', '#351800']
# colorDomain = ['High/Medium', 'Low/Medium']
colorDomain = ['L/M', 'H/M']
y_scale = alt.Scale(domain=[0, 10], clamp=True)

line2 = alt.Chart(
    d
).transform_calculate(
#     order="{'High/Medium':0, 'Low/Medium': 1}[datum.variable]"  
    order="{'H/M':0, 'L/M': 1}[datum.variable]"  
).transform_filter(
    {'field': 'metric', 'oneOf': [metric]}
).encode(
#     x=alt.X("month:N", title=None, axis=alt.Axis(labelAngle=-55), sort=date),
    x=alt.X("month:N", title=None, scale=alt.Scale(domain=date), axis=alt.Axis(grid=False,labelAngle=-55)),
#     x=alt.X("month:N", title='Month', scale=alt.Scale(domain=date), axis=alt.Axis(grid=True, ticks=False, labels=False, domain=False, title=None)),
    y=alt.Y("value:Q", title=y_title, scale=y_scale),
    color=alt.Color("cat:N", title='Risk Ratio', scale=alt.Scale(domain=colorDomain, range=colors)),
    order="order:O"
).properties(
    width=width,
    height=height2
)


line2 = line2.mark_line(
    size=3, point=point, opacity=0.8
).facet(
    column=alt.Column('site:N', header=alt.Header(title=None, labels=False), sort=sites),
)


# Just to show color legend and left axis
# line_not_first = line
# line_not_first = line.encode(
#     x=alt.X("month:N", title=None, axis=alt.Axis(labelAngle=-55), sort=date),
#     y=alt.Y("value:Q", title=None, axis=alt.Axis(format='.0%', ticks=False, labels=False, domain=False), scale=y_scale),
#     color=alt.Color("cat:N", title='Risk Level', scale=alt.Scale(domain=colorDomain, range=colors), legend=None),
#     order="order:O"
# )

# line2_first = line2.encode(
#     x=alt.X("month:N", title=None, sort=date),
#     y=alt.Y("value:Q", title=None, axis=alt.Axis(format='.0%', ticks=False, labels=False, domain=False), scale=y_scale),
#     color=alt.Color("cat:N", title='Risk Ratio (Right Y Axis)', scale=alt.Scale(domain=colorDomain, range=colors)),
#     order="order:O"
# )
# line2_last = line2.encode(
#     x=alt.X("month:N", title=None, sort=date),
#     y=alt.Y("value:Q", title=y_title, axis=alt.Axis(format='.0%'), scale=y_scale),
#     color=alt.Color("cat:N", title='Risk Ratio', scale=alt.Scale(domain=colorDomain, range=colors), legend=None),
#     order="order:O"
# )

# line = alt.concat(*(
#     alt.layer(line, line2_first, title={
#         "text": site,
#         "fontSize": 16,
#         "dx": 130}).transform_filter(alt.datum.site == site).resolve_scale(y='independent', color='independent') if site == 'META' else 
    
#     alt.layer(line_not_first, line2_last, title={
#         "text": site,
#         "fontSize": 16,
#         "dx": 85}).transform_filter(alt.datum.site == site).resolve_scale(y='independent', color='independent') if site == sites[-1] else 
    
#     alt.layer(line_not_first, line2, title={
#         "text": site,
#         "fontSize": 16,
#         "dx": 85}).transform_filter(alt.datum.site == site).resolve_scale(y='independent', color='independent')
#     for site in sites
# ), spacing=3).resolve_scale(color='shared')

print(d.site.unique())

"""
COMBINE
"""
top = bar.properties(
    title={
        "text": [
            f"Distribution Of Parients By Risk Level"
        ],
        "fontWeight": "normal",
        "dx": 170,
        "subtitle": [
            get_visualization_subtitle(data_release=data_release, with_num_sites=False)
        ], 
        "subtitleColor": "gray",
    }
)

bot = line.properties(
    title={
        "text": [
            f"Event Rate Of Risk Scores"
        ],
        "dx": 150,
        "subtitle": [
            get_visualization_subtitle(data_release=data_release, with_num_sites=False)
        ], 
        "subtitleColor": "gray",
    }
)

# top.display()
# bot.display()
# line2.display()
res = alt.vconcat(
    top, 
    bot, 
    line2, 
    spacing=10
).resolve_scale(y='independent', color='independent')

"""
STYLE
"""

res = apply_theme(
    res,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='left',
#     legend_title_orient='left',
    axis_label_font_size=14,
    header_label_font_size=16,
    point_size=90
)

res

In [ ]:
d = df.copy()

d = d[d.site == 'META']

width = 200
height = 200
size = 30
point=alt.OverlayMarkDef(filled=False, fill='white', strokeWidth=2)


"""
DATA PREPROCESSING...
"""
d.loc[d.site == 'combine', 'site'] = 'All Sites'
d.cat = d.cat.apply(lambda x: {
    'L':'Low', 
    'M': 'Medium', 
    'H': 'High',
    'H/M': 'H/M',
    'L/M': 'L/M'
}[x])



"""
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%% TOP %%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
"""
metric='pos'

y_title = '% of Patients in Each Category'
y_scale = alt.Scale(domain=[0, 1])
colors = ['#7BADD1', '#427BB5', '#14366E']
# colorDomain = ['Low Risk', 'Medium Risk', 'High Risk']
colorDomain = ['Low', 'Medium', 'High']
# colorDomain = ['L', 'M', 'H']

bar = alt.Chart(
    d
).transform_calculate(
    order="{'L':0, 'M': 1, 'H': 2}[datum.variable]"  
).transform_filter(
    {'field': 'metric', 'oneOf': [metric]}
).encode(
    x=alt.X("month:N", title=None, axis=alt.Axis(labelAngle=-55), sort=date),
    y=alt.Y("value:Q", title=y_title, axis=alt.Axis(format='.0%'), scale=y_scale),
    color=alt.Color("cat:N", title='Risk Level', scale=alt.Scale(domain=colorDomain, range=colors)),
    order="order:O"
).properties(
    width=width,
    height=height
)

bar = bar.mark_bar(
    size=size, stroke='black'
)


d['visibility'] = d['value'] > 0.08

text = alt.Chart(
    d
).transform_filter(
    {'field': 'metric', 'oneOf': [metric]}
).mark_text(size=16, dx=0, dy=5, color='white', baseline='top', fontWeight=500).encode(
    x=alt.X("month:N", title=None, axis=alt.Axis(labelAngle=-55), sort=date),
    y=alt.Y('value:Q', stack='zero'),
    detail='cat:N',
    text=alt.Text('value:Q', format='.0%'),
    order="order:O",
    opacity=alt.Opacity('visibility:N', scale=alt.Scale(domain=[True, False], range=[1, 0]), legend=None)
)

if metric == 'pos':
    bar = (bar + text)

# bar = bar.facet(
#     column=alt.Column('site:N', header=alt.Header(title=None), sort=sites),
#     spacing=20
# )

"""
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%% Bottom %%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
"""
metric='ppv'

y_title = '% of Event'
colors = ['#A8DED1', '#3FA86F', '#005A24']
colors = ['#00A87E', '#00634B', 'black']
y_scale = alt.Scale(domain=[0, 1])

line = alt.Chart(
    d
).transform_calculate(
#     order="{'Low Risk':0, 'Medium Risk': 1, 'High Risk': 2}[datum.variable]"  
    order="{'L':0, 'M': 1, 'H': 2}[datum.variable]"  
).transform_filter(
    {'field': 'metric', 'oneOf': [metric]}
).encode(
    # x=alt.X("month:N", title=None, axis=alt.Axis(labelAngle=-55), sort=date),
    x=alt.X("month:N", title=None, scale=alt.Scale(domain=date), axis=alt.Axis(labelAngle=-55)),
    y=alt.Y("value:Q", title=y_title, scale=y_scale),
    color=alt.Color("cat:N", title='Risk Level', scale=alt.Scale(domain=colorDomain, range=colors)),
    order="order:O"
).properties(
    width=width,
    height=height
)


line = line.mark_line(
    size=3, point=point, opacity=0.8
)


"""
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%% Bottom 2 %%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
"""
metric='rr'
y_title = 'Ratio Between Risk Levels'
# colors = ['red', 'salmon']
colors = ['#D45E00', '#351800']
# colorDomain = ['High/Medium', 'Low/Medium']
colorDomain = ['L/M', 'H/M']
y_scale = alt.Scale(domain=[0, 10],  clamp=True)

line2 = alt.Chart(
    d
).transform_calculate(
#     order="{'High/Medium':0, 'Low/Medium': 1}[datum.variable]"  
    order="{'H/M':0, 'L/M': 1}[datum.variable]"  
).transform_filter(
    {'field': 'metric', 'oneOf': [metric]}
).encode(
#     x=alt.X("month:N", title=None, axis=alt.Axis(labelAngle=-55), sort=date),
    x=alt.X("month:N", title=None, scale=alt.Scale(domain=date), axis=alt.Axis(grid=False,labelAngle=-55)),
#     x=alt.X("month:N", title='Month', scale=alt.Scale(domain=date), axis=alt.Axis(grid=True, ticks=False, labels=False, domain=False, title=None)),
    y=alt.Y("value:Q", title=y_title, scale=y_scale),
    color=alt.Color("cat:N", title='Risk Ratio', scale=alt.Scale(domain=colorDomain, range=colors)),
    order="order:O"
).properties(
    width=width,
    height=height
)


line2 = line2.mark_line(
    size=3, point=point, opacity=0.8
)

print(d.site.unique())

"""
COMBINE
"""
res = alt.hconcat(
    bar, 
    line, 
    line2, 
    spacing=30
).resolve_scale(y='independent', color='independent')

res = res.properties(
    title={
        "text": [
            f"Meta-Analysis Of Risk Scores"
        ],
        "dx": 70,
#         "subtitle": [
#             get_visualization_subtitle(data_release=data_release, with_num_sites=False)
#         ], 
        "subtitleColor": "gray",
    }
)

"""
STYLE
"""

res = apply_theme(
    res,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='top',
#     legend_title_orient='left',
    axis_label_font_size=14,
    header_label_font_size=16,
    point_size=90,
)

res

# Deprecated

In [ ]:
pos = risk(df, metric='pos')
ppv = risk(df, metric='ppv')

res = alt.vconcat(pos, ppv, spacing=30).resolve_scale(color='independent', x='independent')

res = apply_theme(
    res,
    axis_y_title_font_size=16,
    title_anchor='start',
    legend_orient='right',
    header_label_font_size=16
)

res.display()

In [ ]:
d = df.copy()

width = 280
height = 200
height2 = 140
size = 50


"""
DATA PREPROCESSING...
"""
d.loc[d.site == 'combine', 'site'] = 'All Sites'
d.cat = d.cat.apply(lambda x: {
    'L':'Low', 
    'M': 'Medium', 
    'H': 'High',
    'H/M': 'H/M',
    'L/M': 'L/M'
}[x])
# d.cat = d.cat.apply(lambda x: {
#     'L':'Low Risk', 
#     'M': 'Medium Risk', 
#     'H': 'High Risk',
#     'H/M': 'High/Medium',
#     'L/M': 'Low/Medium'
# }[x])




"""
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%% TOP %%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
"""
metric='pos'

y_title = '% of Patients in Each Category'
y_scale = alt.Scale(domain=[0, 1])
colors = ['#7BADD1', '#427BB5', '#14366E']
# colorDomain = ['Low Risk', 'Medium Risk', 'High Risk']
colorDomain = ['Low', 'Medium', 'High']
# colorDomain = ['L', 'M', 'H']

bar = alt.Chart(
    d
).transform_calculate(
#     order="{'Low Risk':0, 'Medium Risk': 1, 'High Risk': 2}[datum.variable]"  
    order="{'L':0, 'M': 1, 'H': 2}[datum.variable]"  
).transform_filter(
    {'field': 'metric', 'oneOf': [metric]}
).encode(
    x=alt.X("month:N", title='Month', scale=alt.Scale(domain=['Mar-Apr', 'May-Jun', 'Jul-Aug', 'Sep-Oct', 'Since Nov']), axis=alt.Axis(grid=True)),
    y=alt.Y("value:Q", title=y_title, axis=alt.Axis(format='.0%'), scale=y_scale),
    color=alt.Color("cat:N", title='Risk', scale=alt.Scale(domain=colorDomain, range=colors)),
    order="order:O"
).properties(
    width=width,
    height=height
)

bar = bar.mark_bar(
    size=size, stroke='black'
)


d['visibility'] = d['value'] > 0.08

text = alt.Chart(
    d
).transform_filter(
    {'field': 'metric', 'oneOf': [metric]}
).mark_text(size=16, dx=0, dy=5, color='white', baseline='top', fontWeight=500).encode(
    x=alt.X('month:N'),
    y=alt.Y('value:Q', stack='zero'),
    detail='cat:N',
    text=alt.Text('value:Q', format='.0%'),
    order="order:O",
    opacity=alt.Opacity('visibility:N', scale=alt.Scale(domain=[True, False], range=[1, 0]), legend=None)
)

if metric == 'pos':
    bar = (bar + text)

bar = bar.facet(
    column=alt.Column('site:N', header=alt.Header(title=None)),
)

"""
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%% Bottom %%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
"""
metric='ppv'

y_title = '% of Risk Event'
colors = ['#A8DED1', '#3FA86F', '#005A24']
colors = ['#00A87E', '#00634B', 'black']
y_scale = alt.Scale(domain=[0, 0.6])

line = alt.Chart(
    d
).transform_calculate(
#     order="{'Low Risk':0, 'Medium Risk': 1, 'High Risk': 2}[datum.variable]"  
    order="{'L':0, 'M': 1, 'H': 2}[datum.variable]"  
).transform_filter(
    {'field': 'metric', 'oneOf': [metric]}
).encode(
    x=alt.X("month:N", title='Month', scale=alt.Scale(domain=['Mar-Apr', 'May-Jun', 'Jul-Aug', 'Sep-Oct', 'Since Nov']), axis=alt.Axis(grid=True, ticks=False, labels=False, domain=False, title=None)),
    y=alt.Y("value:Q", title=y_title, axis=alt.Axis(format='.0%'), scale=y_scale),
    color=alt.Color("cat:N", title='Risk', scale=alt.Scale(domain=colorDomain, range=colors)),
    order="order:O"
).properties(
    width=width,
    height=height2
)


line = line.mark_line(
    size=3, point=True, opacity=0.8
).facet(
    column=alt.Column('site:N', header=alt.Header(title=None)),
)



"""
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%% Bottom 2 %%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
"""
metric='rr'
y_title = 'Ratio Between Risks'
colors = ['#D45E00', '#351800']
# colorDomain = ['High/Medium', 'Low/Medium']
colorDomain = ['L/M', 'H/M']
y_scale = alt.Scale(domain=[0, 4.2])

line2 = alt.Chart(
    d
).transform_calculate(
#     order="{'High/Medium':0, 'Low/Medium': 1}[datum.variable]"  
    order="{'H/M':0, 'L/M': 1}[datum.variable]"  
).transform_filter(
    {'field': 'metric', 'oneOf': [metric]}
).encode(
    x=alt.X("month:N", title='Month', scale=alt.Scale(domain=['Mar-Apr', 'May-Jun', 'Jul-Aug', 'Sep-Oct', 'Since Nov']), axis=alt.Axis(grid=True)),
    y=alt.Y("value:Q", title=y_title, axis=alt.Axis(format='.0%'), scale=y_scale),
    color=alt.Color("cat:N", title='Risk Ratio', scale=alt.Scale(domain=colorDomain, range=colors)),
    order="order:O",
#     shape="site:N"
).properties(
    width=width,
    height=height2
)

line2 = line2.mark_line(
    size=3, opacity=0.8, point=True
).facet(
    column=alt.Column('site:N', header=alt.Header(title=None, labels=False)),
)






"""
COMBINE
"""
top = bar.properties(
    title={
        "text": [
            f"Distribution of Risk Scores"
        ],
        "dx": 180,
        "subtitle": [
            get_visualization_subtitle(data_release=data_release, with_num_sites=False)
        ], 
        "subtitleColor": "gray",
    }
)

line = line.properties(
    title={
        "text": [
            f"Event Rate of Risk Scores"
        ],
        "dx": 180,
        "subtitle": [
            get_visualization_subtitle(data_release=data_release, with_num_sites=False)
        ], 
        "subtitleColor": "gray",
    }
)

# line2 = line2.properties(
#     title={
#         "text": [
#             f"Risk Ratio"
#         ],
#         "dx": 180,
#         "subtitle": [
#             get_visualization_subtitle(data_release=data_release, with_num_sites=False)
#         ], 
#         "subtitleColor": "gray",
#     }
# )

res = alt.vconcat(top, line, line2, spacing=10).resolve_scale(color='independent')

"""
STYLE
"""

res = apply_theme(
    res,
    axis_y_title_font_size=14,
    axis_title_font_size=14,
    axis_label_font_size=12,
    title_anchor='start',
    legend_orient='left',
    header_label_font_size=16
)

res